In [2]:
import sqlite3
from tokens import TOKENVK
import pandas as pd
import requests
import datetime

con = sqlite3.connect("VK_page_testcase2.db")

In [ ]:
# Парсер постов со страницы с использованием VK API
TOKEN_USER = TOKENVK
VERSION = 5.199
OWNER_ID = "ltkitten"

response = requests.get('https://api.vk.com/method/wall.get',
params={'access_token': TOKEN_USER,
        'v': VERSION,
        'count': 100,
        'owner_id': OWNER_ID,
        'filter': str('owner')})

data = response.json()['response']['items']
print(data[0])

In [162]:
# Создаем Список для дальнейшей загрузки в таблицу.
data_to_sql = []
for i in range(len(data)):
    temp_list = []
    temp_list.append(data[i]['likes']['count'])
    temp_list.append(data[i]['date'])
    data_to_sql.append(temp_list)
    data_to_sql[i][1] = datetime.datetime.fromtimestamp(data_to_sql[i][1])

In [4]:
# Создаем подключение к движку sqlite
cur = con.cursor()

In [164]:
# Создаем таблицу и выполняем запрос
cur.execute("CREATE TABLE VkPosts(likes, Date, Daypart, Weekday)")
con.commit()

In [165]:
# Создаем столбец времени суток для аналитики
for i in range(len(data_to_sql)):
    if data_to_sql[i][1].time() <= datetime.time(5, 59, 59):
        data_to_sql[i].append("Night")
        continue
    if data_to_sql[i][1].time() <= datetime.time(11, 59, 59):
        data_to_sql[i].append("Morning")
        continue
    if data_to_sql[i][1].time() <= datetime.time(17, 59, 59):
        data_to_sql[i].append("Afternoon")
        continue
    if data_to_sql[i][1].time() <= datetime.time(23, 59, 59):
        data_to_sql[i].append("Evening")
        continue

In [ ]:
# Создаем столбец дня недели для аналитики
for i in range(len(data_to_sql)):
    if data_to_sql[i][1].weekday() == 0:
        data_to_sql[i].append('Monday')
        continue
    if data_to_sql[i][1].weekday() == 1:
        data_to_sql[i].append('Tuesday')
        continue
    if data_to_sql[i][1].weekday() == 2:
        data_to_sql[i].append('Wednesday')
        continue
    if data_to_sql[i][1].weekday() == 3:
        data_to_sql[i].append('Thursday')
        continue
    if data_to_sql[i][1].weekday() == 4:
        data_to_sql[i].append('Friday')
        continue
    if data_to_sql[i][1].weekday() == 5:
        data_to_sql[i].append('Saturday')
        continue
    if data_to_sql[i][1].weekday() == 6:
        data_to_sql[i].append('Sunday')
        continue
for i in range(len(data_to_sql)):
    print(data_to_sql[i])

In [167]:
# Загружаем спарсенные данные в таблицу
insert_query = ("INSERT INTO VkPosts(likes, Date, Daypart, Weekday) VALUES (?, ?, ?, ?)")
cur.executemany(insert_query, data_to_sql)
con.commit()

In [59]:
cur.close()

In [ ]:
# Начинаем писать сами запросы.
# Запрос 1. Влияет ли время суток на количество лайков?
cur.execute("SELECT SUM(likes), Daypart FROM VkPosts GROUP BY Daypart")
sqldata=cur.fetchall()
print(sqldata)

In [ ]:
# Запрос 2. Влияет ли день недели на количество лайков?
cur.execute("SELECT SUM(likes), Weekday FROM VkPosts WHERE likes GROUP BY Weekday")
sqldata=cur.fetchall()
print(sqldata)

In [ ]:
# Запрос 3. Влияет ли промежуток?
cur.execute('''SELECT
    likes,
    Date,
    JULIANDAY(Date) - JULIANDAY(LAG(Date) OVER (ORDER BY Date)) AS day_diff
FROM
    VkPosts
ORDER BY
    Date DESC;''')
sqldata=cur.fetchall()
print(sqldata)

In [ ]:
# Задание 3. Уберем выбросы в ллайках на моей странице.
cur.execute('''SELECT
    likes,
    Date,
    JULIANDAY(Date) - JULIANDAY(LAG(Date) OVER (ORDER BY Date)) AS day_diff
FROM
    VkPosts
WHERE likes < 25
ORDER BY
    Date DESC;''')
sqldata=cur.fetchall()
print(sqldata)

In [139]:
db_df = pd.read_sql_query("SELECT * FROM VkPosts", con)
db_df.to_csv('database2.csv', index=False)

In [156]:
#Закрываем коннект
con.close()